In [60]:
import edgar
import numpy as np
import pandas as pd
import requests
import bs4

In [61]:
#Download all filings since 2021
# edgar.download_index('filings', 2021, user_agent="Michael michael20010808@gmail.com", skip_all_present_except_last=False)

In [62]:
# lower case and all small letters

selected_company = 'American Airlines Group Inc'.lower()
ticker = 'aal'
selected_report = '10-Q'

In [63]:
# Get and Format the dataframe for query in the future

csv = pd.read_csv('filings/2022-QTR2.tsv', sep='\t', lineterminator='\r', names=None)



csv.columns = ['Item']
csv['company'] = csv['Item'].str.split('|').str[1].str.lower()
csv['company'] = csv['company'].apply(lambda x: str(x)[:-1] if str(x)[-1] == '.' else str(x))

In [64]:
company_report = csv[(csv['company'].str.contains(selected_company)) & (csv['Item'].str.contains(selected_report))]
filing = company_report['Item'].str.split('|').to_list()
Q_reports_lst = []
for report in filing:
    Q_reports_lst.append(report[-1])
Q_reports_lst

['edgar/data/6201/0000006201-22-000046-index.html']

In [65]:
report_1 = Q_reports_lst[0]
url = 'https://www.sec.gov/Archives/' + report_1
header = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
r = requests.get(url, headers=header)
df = pd.read_html(r.text)



In [66]:
document_index = df[0]
document_index = document_index.dropna()
document_name = document_index[document_index['Description'].str.contains(selected_report)]
document_name = document_name['Document'].str.split(' ')[0][0]
report_formatted = report_1.replace('-', '').replace('index.html','')
url = 'https://www.sec.gov/Archives/' + report_formatted + '/' + document_name
url

'https://www.sec.gov/Archives/edgar/data/6201/000000620122000046/aal-20220331.htm'

In [115]:
r2 = requests.get(url, headers=header)
df = pd.read_html(r2.text)


In [116]:
# Measure in thousands or millions

soup = bs4.BeautifulSoup(r2.text, 'html.parser')
matched = soup.find_all('span')

for match in matched:
    if ' in thousands' in match.text:
        measured_base = 'thousands'
        measure_multiple = 1000
        break
    elif 'in millions' in match.text:
        measured_base = 'millions'
        measure_multiple = 1000000
        break

In [126]:
found = 0
for item in df:
    try:
        print(item)
        # search in the dataframe item that if there is cell with word 'statement of operations' in it
        for col in item.columns.values:
            print(item[col])
            found_IS = item[col].apply(lambda x: True if 'statements of operations' in x.lower() else False, axis=0)
            if found_IS.any():
                found = 1
                break
        if item[0].shape[0] > 10 and item[0].isna().sum()/item[0].shape[0] < 0.3:
            BS_1 = (item[0].str.lower().str.contains('total current liabilities'))
            BS_2 = (item[0].str.lower().str.contains('total assets'))
            IS_1 = (item[0].str.lower().str.contains('total revenue') | item[0].str.lower().str.contains('total net sales'))
            IS_2 = (item[0].str.lower().str.contains('interest expense'))
            CF = (item[0].str.lower().str.contains('cash flows from operating activities') |
                  item[0].str.lower().str.contains('cash flows from investing activities') |
                  item[0].str.lower().str.contains('cash flows from financing activities') |
                  item[0].str.lower().str.contains('operating activities') |
                  item[0].str.lower().str.contains('financing activities') |
                  item[0].str.lower().str.contains('investing activities') )
            if BS_1.any() and BS_2.any():
                Balance_Sheet = item
            elif (IS_1.any() and IS_2.any() and '$' in item.iloc[:, 1:].values) or found==1:
                Income_Statement = item
                found=0
            elif CF.any():
                Cash_flow = item
    except:
        pass

      0     1     2     3     4     5
0   NaN   NaN   NaN   NaN   NaN   NaN
1   NaN   NaN   NaN   NaN   NaN   NaN
2  FORM  FORM  FORM  10-Q  10-Q  10-Q
3   NaN   NaN   NaN   NaN   NaN   NaN
0     NaN
1     NaN
2    FORM
3     NaN
Name: 0, dtype: object
     0    1    2                                                  3  \
0  NaN  NaN  NaN                                                NaN   
1    ☒    ☒    ☒  Quarterly Report Pursuant to Section 13 or 15(...   

                                                   4  \
0                                                NaN   
1  Quarterly Report Pursuant to Section 13 or 15(...   

                                                   5  
0                                                NaN  
1  Quarterly Report Pursuant to Section 13 or 15(...  
0    NaN
1      ☒
Name: 0, dtype: object
     0    1    2                                                  3  \
0  NaN  NaN  NaN                                                NaN   
1    ☐    ☐    ☐

In [70]:
# drop columns with more than 70% of missing values
# So we keep the values only

Balance_Sheet = Balance_Sheet.dropna(axis=1, thresh=int(Balance_Sheet.shape[0]*0.3))

# drop cols with signs
for col in Balance_Sheet.columns:
    if ('$' in Balance_Sheet[col].values) | (')' in Balance_Sheet[col].values):
        Balance_Sheet = Balance_Sheet.drop(col, axis=1)

# drop duplicates columns
Balance_Sheet = Balance_Sheet.T.drop_duplicates().T
Balance_Sheet.columns = range(Balance_Sheet.shape[1])
Balance_Sheet

,0,1,2
0,NaN,NaN,NaN
1,NaN,"March 31, 2022","December 31, 2021"
2,NaN,(Unaudited),NaN
3,ASSETS,NaN,NaN
4,Current assets,NaN,NaN
5,Cash,361,265
6,Short-term investments,12093,12155
7,Restricted cash and short-term investments,952,990
8,"Accounts receivable, net",1516,1484
9,"Receivables from related parties, net",5954,5547


In [71]:
# Get The Balance Sheet

# Get the right columns
Balance_Sheet_cleaned = Balance_Sheet.iloc[Balance_Sheet.iloc[:,0].first_valid_index():, :]
string = Balance_Sheet.iloc[Balance_Sheet.iloc[:,1].first_valid_index():Balance_Sheet.iloc[:,0].first_valid_index(),1:]
new_cols = ['Item']

Balance_Sheet_cleaned = Balance_Sheet_cleaned.reset_index(drop=True)

for i in range(1, Balance_Sheet_cleaned.shape[1]):
    new_cols.append(string.iloc[:,i-1].str.cat(sep=' '))

    # Format the values
    Balance_Sheet_cleaned.loc[:, i] = Balance_Sheet_cleaned.loc[:, i].astype(str).values

    Balance_Sheet_cleaned.iloc[:, i] = Balance_Sheet_cleaned.iloc[:, i].map(lambda x: x.replace('(', '-').replace(')','').replace(',', '').replace('—', '0'))

    Balance_Sheet_cleaned.loc[:, i] = Balance_Sheet_cleaned.loc[:, i].astype(float).values

Balance_Sheet_cleaned.columns = new_cols
Balance_Sheet_cleaned['Item'] = Balance_Sheet_cleaned['Item'].astype(str)
Balance_Sheet_cleaned = Balance_Sheet_cleaned.iloc[:,:3]
Balance_Sheet_cleaned



,Item,"March 31, 2022 (Unaudited)","December 31, 2021"
0,ASSETS,NaN,NaN
1,Current assets,NaN,NaN
2,Cash,361.0,265.0
3,Short-term investments,12093.0,12155.0
4,Restricted cash and short-term investments,952.0,990.0
5,"Accounts receivable, net",1516.0,1484.0
6,"Receivables from related parties, net",5954.0,5547.0
7,"Aircraft fuel, spare parts and supplies, net",1897.0,1692.0
8,Prepaid expenses and other,748.0,579.0
9,Total current assets,23521.0,22712.0


In [72]:
# drop columns with more than 70% of missing values
# So we keep the values only

Cash_flow = Cash_flow.dropna(axis=1, thresh=int(Cash_flow.shape[0]*0.3))

# drop cols with signs
for col in Cash_flow.columns:
    if ('$' in Cash_flow[col].values) | (')' in Cash_flow[col].values):
        Cash_flow = Cash_flow.drop(col, axis=1)

# drop duplicates columns
Cash_flow = Cash_flow.T.drop_duplicates().T
Cash_flow.columns = range(Cash_flow.shape[1])
Cash_flow

,0,1,2
0,NaN,NaN,NaN
1,NaN,"Three Months Ended March 31,","Three Months Ended March 31,"
2,NaN,2022,2021
3,Net cash provided by operating activities,810,1428
4,Cash flows from investing activities:,NaN,NaN
5,"Capital expenditures, net of aircraft purchase...",(790),24
6,"Airport construction projects, net of reimburs...",(62),(42)
7,Proceeds from sale of property and equipment,8,108
8,Proceeds from sale-leaseback transactions,—,99
9,Sales of short-term investments,7089,1415


In [73]:
# Get The Cash Flow

# Get the right columns
Cash_flow_cleaned = Cash_flow.iloc[Cash_flow.iloc[:,0].first_valid_index():, :]
string = Cash_flow.iloc[Cash_flow.iloc[:,1].first_valid_index():Cash_flow.iloc[:,0].first_valid_index(),1:]
new_cols = ['Item']

Cash_flow_cleaned = Cash_flow_cleaned.reset_index(drop=True)

for i in range(1, Cash_flow_cleaned.shape[1]):
    new_cols.append(string.iloc[:,i-1].str.cat(sep=' '))

    # Format the values
    Cash_flow_cleaned.loc[:, i] = Cash_flow_cleaned.loc[:, i].astype(str).values

    Cash_flow_cleaned.iloc[:, i] = Cash_flow_cleaned.iloc[:, i].map(lambda x: x.replace('(', '-').replace(')','').replace(',', '').replace('—', '0'))

    Cash_flow_cleaned.loc[:, i] = Cash_flow_cleaned.loc[:, i].astype(float).values

Cash_flow_cleaned.columns = new_cols
Cash_flow_cleaned['Item'] = Cash_flow_cleaned['Item'].astype(str)
Cash_flow_cleaned = Cash_flow_cleaned.iloc[:,:3]
Cash_flow_cleaned

,Item,"Three Months Ended March 31, 2022","Three Months Ended March 31, 2021"
0,Net cash provided by operating activities,810.0,1428.0
1,Cash flows from investing activities:,NaN,NaN
2,"Capital expenditures, net of aircraft purchase...",-790.0,24.0
3,"Airport construction projects, net of reimburs...",-62.0,-42.0
4,Proceeds from sale of property and equipment,8.0,108.0
5,Proceeds from sale-leaseback transactions,0.0,99.0
6,Sales of short-term investments,7089.0,1415.0
7,Purchases of short-term investments,-7022.0,-8545.0
8,Decrease (increase) in restricted short-term i...,36.0,-194.0
9,nan,NaN,NaN


In [74]:
# drop columns with more than 70% of missing values
# So we keep the values only

Income_Statement = Income_Statement.dropna(axis=1, thresh=int(Cash_flow.shape[0]*0.3))

# drop cols with signs
for col in Income_Statement.columns:
    if ('$' in Income_Statement[col].values) | (')' in Income_Statement[col].values): Income_Statement = Income_Statement.drop(col, axis=1)

# drop duplicates columns
Income_Statement = Income_Statement.T.drop_duplicates().T
Income_Statement.columns = range(Income_Statement.shape[1])
Income_Statement

,0,1,2
0,NaN,NaN,NaN
1,NaN,Three Months Ended,Three Months Ended
2,NaN,"April 30,","May 1,"
3,NaN,2022,2021
4,REVENUES:,NaN,NaN
5,Net sales,1925643,2190667
6,Other revenue,4049,2629
7,Total revenue,1929692,2193296
8,COSTS AND EXPENSES:,NaN,NaN
9,Cost of sales,1136946,1242189


In [75]:
# Get The Income Statement

# Get the right columns
Income_Statement_cleaned = Income_Statement.iloc[Income_Statement.iloc[:,0].first_valid_index():, :]
string = Income_Statement.iloc[Income_Statement.iloc[:,1].first_valid_index():Income_Statement.iloc[:,0].first_valid_index(),1:]
new_cols = ['Item']

Income_Statement_cleaned = Income_Statement_cleaned.reset_index(drop=True)

for i in range(1, Income_Statement_cleaned.shape[1]):
    new_cols.append(string.iloc[:,i-1].str.cat(sep=' '))

    # Format the values
    Income_Statement_cleaned.loc[:, i] = Income_Statement_cleaned.loc[:, i].astype(str).values

    Income_Statement_cleaned.iloc[:, i] = Income_Statement_cleaned.iloc[:, i].map(lambda x: x.replace('(', '-').replace(')','').replace(',', '').replace('—', '0'))

    Income_Statement_cleaned.loc[:, i] = Income_Statement_cleaned.loc[:, i].astype(float).values

Income_Statement_cleaned.columns = new_cols
Income_Statement_cleaned['Item'] = Income_Statement_cleaned['Item'].astype(str)
Income_Statement_cleaned = Income_Statement_cleaned.iloc[:,:3]
Income_Statement_cleaned

,Item,"Three Months Ended April 30, 2022","Three Months Ended May 1, 2021"
0,REVENUES:,NaN,NaN
1,Net sales,1925643.00,2190667.00
2,Other revenue,4049.00,2629.00
3,Total revenue,1929692.00,2193296.00
4,COSTS AND EXPENSES:,NaN,NaN
5,Cost of sales,1136946.00,1242189.00
6,"Selling, general and administrative expenses",680327.00,664828.00
7,Depreciation and amortization,66304.00,55610.00
8,Impairment charges - long-lived assets,2543.00,777.00
9,Other income - net,-3398.00,-1374.00


In [76]:
# Get Outstanding Debt, Preferred Stock, Minority Interest and Cash and cash equivalents

Short_term_debt = Balance_Sheet_cleaned.loc[Balance_Sheet_cleaned['Item'].str.lower() == 'total current liabilities', :].values[0,1]
Long_term_debt_idx = Balance_Sheet_cleaned['Item'].apply(lambda x: ('long term debt' == x.lower().split(',')[0]) | ('long-term debt' == x.lower().split(',')[0])).idxmax()
Long_term_debt = Balance_Sheet_cleaned.iloc[Long_term_debt_idx, 1]
Out_standing_debt = (Short_term_debt + Long_term_debt) * measure_multiple

Preferred_stock_idx = Balance_Sheet_cleaned['Item'].apply(lambda x: 'preferred stock' in x.lower())
if Preferred_stock_idx.sum() == 0:
    Preferred_stock = 0
else:
    Preferred_stock = Balance_Sheet_cleaned.iloc[Preferred_stock_idx.idxmax(), 1] * measure_multiple
if np.isnan(Preferred_stock):
    Preferred_stock = 0

minority_interest_idx = Balance_Sheet_cleaned['Item'].apply(lambda x: ('minority interest' in x.lower()) | ('noncontrolling interests' in x.lower()))
if minority_interest_idx.sum() == 0:
    minority_interest = 0
else:
    minority_interest = Balance_Sheet_cleaned.iloc[minority_interest_idx.idxmax(), 1] * measure_multiple
if np.isnan(minority_interest):
    minority_interest = 0

Cash_idx = Balance_Sheet_cleaned['Item'].apply(lambda x: ('cash and cash equivalents' == x.lower()) | ('cash' == x.lower()))
if Cash_idx.sum() == 0:
    Cash = 0
else:
    Cash = Balance_Sheet_cleaned.iloc[Cash_idx.idxmax(), 1] * measure_multiple
if np.isnan(Cash):
    Cash = 0



In [77]:
# MARkET CAP

mk_cap_link = f"https://finance.yahoo.com/quote/{ticker.upper()}?p=B{ticker.upper()}&.tsrc=fin-srch"
r = requests.get(mk_cap_link, headers=header)
soup = bs4.BeautifulSoup(r.text, 'html.parser')

mkt_cap = soup.find('td', attrs={'class':'Ta(end) Fw(600) Lh(14px)', 'data-test':"MARKET_CAP-value"}).text


In [78]:
if mkt_cap[-1] == 'T':
    mkt_cap = float(mkt_cap[:-1]) * 10 ** 12
elif mkt_cap[-1] == 'B':
    mkt_cap = float(mkt_cap[:-1]) * 1000000000
elif mkt_cap[-1] == 'M':
    mkt_cap = float(mkt_cap[:-1]) * 1000000
elif mkt_cap[-1] == 'K':
    mkt_cap = float(mkt_cap[:-1]) * 1000
else:
    mkt_cap = float(mkt_cap)

mkt_cap

9280000000.0

In [85]:
# Enterprise Value/ EBITDA
Ev = mkt_cap + Preferred_stock + minority_interest - Cash + Out_standing_debt

total_idx = Income_Statement_cleaned['Item'].apply(lambda x: 'total' in x.lower()).values
total = Income_Statement_cleaned.iloc[total_idx, :2]
EBITDA = (total.iloc[0, 1] - total.iloc[1, 1]) * measure_multiple

EV_EBITDA = Ev / EBITDA

nan